<a href="https://colab.research.google.com/github/hieuducle/SIM-KPDL/blob/main/SIM_Price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [164]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train_dataset.csv")

In [165]:
df

,price_vnd,sim_number
0,450000,343189288
1,3000000,888899580
2,500000,928960006
3,5000000,902438679
4,450000,334307889
...,...,...
199995,450000,866161769
199996,1000000,708124126
199997,1000000,904755200
199998,450000,329220204


In [166]:
df.describe()

,price_vnd,sim_number
count,2.000000e+05,2.000000e+05
mean,1.395027e+07,7.304797e+08
std,5.912997e+08,2.406479e+08
min,9.900000e+04,3.250098e+08
25%,5.000000e+05,3.922605e+08
50%,1.000000e+06,8.334287e+08
75%,5.000000e+06,9.180874e+08
max,1.680000e+11,9.979797e+08


In [167]:
df[df["price_vnd"]<10000000]["price_vnd"].value_counts()

450000     45119
1000000    38472
500000     30619
3000000    29199
5000000    21383
399000       138
299000        91
99000         58
250000        47
119000        32
350000        22
199000        18
290000         9
390000         3
280000         3
400000         2
220000         1
Name: price_vnd, dtype: int64

In [168]:
df[df["price_vnd"]>10000000]["price_vnd"].value_counts()

12000000     5124
11325000     2682
11000000     1760
13000000     1720
15000000     1417
             ... 
130500000       1
12150000        1
80100000        1
424000000       1
439000000       1
Name: price_vnd, Length: 921, dtype: int64

In [169]:
def to_label(x):
  #symmetric
    sim = str(x['sim_number'])
    def is_symmetric(s):
        return s == s[::-1]

    sim_len = len(sim)
    symmetric_label = 3
    stop = False

    for i in range(9):
        s_len = sim_len - i - 1
        j = 0
      
        while j + s_len <= 9 and symmetric_label == 3:
            if is_symmetric(sim[j:j+s_len]):
                if s_len == 8:
                    symmetric_label = 12
                    stop = True
                    break
                
                if s_len == 7:
                    symmetric_label = 11
                    stop = True
                    break
                
                if s_len == 6:
                    symmetric_label = 10
                    stop = True
                    break
                
                if s_len == 5:
                    symmetric_label = 9
                    stop = True
                    break
                
                if s_len == 4:
                    symmetric_label = 8
                    stop = True
                    break
                if s_len == 3:
                    symmetric_label = 6
                    stop = True
                    break
            if stop: break
            j = j + 1 
   
  # tail_number 
    tail_label = 3
   
    five = False
    four = False
    equal5 = ['00000', '11111', '22222', '33333', '44444', 
              '55555', '66666', '77777', '88888', '99999']
    asc5 = ['01234', '12345', '23456', '34567', '45678', '56789']
    desc5 = ['43210', '54321', '65432', '76543', '87654', '98765']
    if str(x['sim_number'])[4:9] in equal5:
      tail_label = 12
      five = True
      four = True

    elif str(x['sim_number'])[4:9] in asc5:
      tail_label = 10
      five = True
      four = True

    elif str(x['sim_number'])[4:9] in desc5:
      tail_label = 10
      five = True
      four = True


    
    if not five:
      equal4 = ['0000', '1111', '2222', '3333', '4444', 
                '5555', '6666', '7777', '8888', '9999']
      asc4 = ['0123', '1234', '2345', '3456', '4567', '5678', '6789']
      desc4 = ['3210', '4321', '5432', '6543', '7654', '8765', '9876']
  
      if str(x['sim_number'])[5:9] in equal4:
        tail_label = 11
        four = True

      elif str(x['sim_number'])[5:9] in asc4:
        tail_label = 8
        four = True

      elif str(x['sim_number'])[5:9] in desc4:
        tail_label = 8
        four = True

    if not four:
      equal3 = ['000', '111', '222', '333', '444',
                '555', '666', '777', '888', '999']
      asc3 = ['123', '234', '345', '456', '567', '678', '789', '012']
      desc3 = ['321', '432', '543', '654', '765', '876', '987', '210']
    
      if str(x['sim_number'])[6:9] in equal3:
        tail_label = 6
      elif str(x['sim_number'])[6:9] in asc3:
        tail_label = 4
      elif str(x['sim_number'])[6:9] in desc3:
        tail_label = 4
    

  

    # price
    price_label =  tail_label
    if x["price_vnd"]<=450000:
      price_label =  0
    elif x["price_vnd"]==500000:
      price_label =  1
    elif x["price_vnd"]==1000000:
      price_label =  2
    elif x["price_vnd"]==3000000:
      price_label =  3
    elif x["price_vnd"]==5000000:
      price_label =  4
    elif x["price_vnd"]==10000000:
      price_label =  5
    elif x["price_vnd"] > 10000000 & x["price_vnd"] <= 12000000:
      price_label =  6
    elif x["price_vnd"] > 12000000 & x["price_vnd"] <= 15000000:
      price_label =  7
    elif x["price_vnd"] > 15000000 & x["price_vnd"] <= 100000000:
      price_label =  8
    elif x["price_vnd"] > 100000000 & x["price_vnd"] <= 500000000:
      price_label =  9
    elif x["price_vnd"] > 500000000 & x["price_vnd"] <= 1000000000:
      price_label =  10
    elif x["price_vnd"] > 1000000000 & x["price_vnd"] <= 1680000000:
      price_label =  11


    return int((price_label + tail_label + symmetric_label) / 3)

df["label"] = df.apply(to_label, axis=1)
# df.loc[len(df.index)] = [] 
  

  

In [204]:
import random
def create_sim():
  list_special_5 = ['00000', '11111', '22222', '33333', '44444', '55555', '66666', '77777', 
                    '88888', '99999','01234', '12345', '23456', '34567', '45678', '56789','43210', 
                    '54321', '65432', '76543', '87654', '98765']

  random_number =  random.randint(0,21)
  s = list_special_5[random_number]
  s_len = len(s)
  sc = ""
  num = random.randint(1,9)
  sc += str(num)
  while len(sc) < 4:
    num = random.randint(0,9)
    sc += str(num)
  s = sc + s
  return s

num_of_sim5 = 1000
for i in range(num_of_sim5):
  sim_number = int(create_sim())
  # print(sim_number)
  label = 10
  price = random.randint(100000000, 500000000)
  df2 = {'price_vnd': price,'sim_number': sim_number, 'label': label}
  df = df.append(df2, ignore_index = True)
  


In [205]:

df

,price_vnd,sim_number,label
0,450000,343189288,3
1,3000000,888899580,4
2,500000,928960006,4
3,5000000,902438679,3
4,450000,334307889,3
...,...,...,...
201995,268418756,845054321,10
201996,116964810,716754321,10
201997,113406820,992254321,10
201998,250148154,659000000,10


In [206]:
df["label"].value_counts()

3     68951
2     44471
4     42660
5     26800
6     12461
8      3324
10     2006
7       791
9       536
Name: label, dtype: int64

In [172]:
X = []
y = []

for index, row in df.iterrows():
  X.append([int(c) for c in str(row['sim_number'])])
  y.append(row["label"])

X, y = np.array(X), np.array(y)

In [173]:
X.shape

(201000, 9)

In [174]:
y.shape

(201000,)

In [176]:
# Split dữ liệu train, test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [141]:
X_train.shape

(180900,)

In [177]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
import keras
optimizer = keras.optimizers.Adam(lr=0.001)



/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [179]:
# The LSTM architecture
regressor = Sequential()
# First LSTM layer with Dropout regularisation
regressor.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],1)))
regressor.add(Dropout(0.2))
# Second LSTM layer
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))
# Third LSTM layer
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))
# Fourth LSTM layer
regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))
# The output layer
regressor.add(Dense(units=1))
# model.add(Dense(3, ))
# Compiling the RNN
regressor.compile(optimizer='rmsprop',loss='mean_squared_logarithmic_error')
# regressor.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
# Fitting to the training set
regressor.summary()
regressor.fit(X_train,y_train,epochs=5,batch_size=128)


Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_76 (LSTM)              (None, 9, 50)             10400     
                                                                 
 dropout_76 (Dropout)        (None, 9, 50)             0         
                                                                 
 lstm_77 (LSTM)              (None, 9, 50)             20200     
                                                                 
 dropout_77 (Dropout)        (None, 9, 50)             0         
                                                                 
 lstm_78 (LSTM)              (None, 9, 50)             20200     
                                                                 
 dropout_78 (Dropout)        (None, 9, 50)             0         
                                                                 
 lstm_79 (LSTM)              (None, 50)              

In [180]:
df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test (1).csv")

In [184]:
X_t = []
for index, row in df_test.iterrows():
  X_t.append([int(c) for c in str(row['sim_number'])])


In [185]:
import random


def label_to_price(y_pred):
  prices = []
  for p in list(y_pred):
    i = int (round(p[0]))
    
    if i == 1:
      prices.append(random.randint(400000,450000))

    elif i == 2:
      prices.append(random.randint(450000,750000))

    elif i == 3:
      prices.append(random.randint(750000,1000000))

    elif i == 4:
      prices.append(random.randint(1000000,3000000))

    elif i == 5:
      prices.append(random.randint(3000000,5000000))

    elif i == 6:
      prices.append(random.randint(5000000,10000000))
    
    elif i == 7:
      prices.append(random.randint(10000000, 12000000))

    elif i == 8:
      prices.append(random.randint(12000000, 15000000))

    elif i == 9:
      prices.append(random.randint(15000000, 100000000))

    elif i == 10:
      prices.append(random.randint(100000000, 500000000))

    elif i == 11:
      prices.append(random.randint(500000000, 1000000000))

    elif i == 12:
      prices.append(random.randint(1000000000, 1680000000))
  return prices

In [187]:
y_pred = regressor.predict(X_t)

4659/4659 [==============================] - 39s 8ms/step


In [203]:
y_pred

array([[3.392655 ],
       [2.8230438],
       [3.5255022],
       ...,
       [2.6515653],
       [3.5605304],
       [3.2844427]], dtype=float32)

In [189]:

price_pred = label_to_price(y_pred)


In [190]:
max(price_pred)

99930664

In [200]:
df_price = pd.DataFrame(price_pred, columns = ["price_pred"])
result = pd.concat([df_test['sim_number'], df_price], axis=1, join='inner')
display(result)
result = pd.DataFrame(result)

,sim_number,price_pred
0,776260543,977577
1,822977488,988386
2,707963979,2976687
3,344134986,671301
4,795718118,755758
...,...,...
149055,822139139,915320
149056,922491999,7232929
149057,332378966,985620
149058,378492223,1332617


In [201]:
result.describe()

,sim_number,price_pred
count,1.490600e+05,1.490600e+05
mean,7.293419e+08,1.889852e+06
std,2.409656e+08,3.801616e+06
min,3.250059e+08,4.500150e+05
25%,3.897185e+08,7.986898e+05
50%,8.298890e+08,9.172300e+05
75%,9.177405e+08,1.779531e+06
max,9.980000e+08,9.993066e+07


In [202]:
result.to_csv(r'/content/drive/MyDrive/Colab Notebooks/result_pred.csv', index=False)

In [198]:
result = regressor.predict(np.expand_dims(X[7484], axis=0))
print(label_to_price(result))
print(X[7484])

1/1 [==============================] - 0s 28ms/step
[1610309]
[7 9 9 1 2 0 2 1 5]
